In [ ]:
# =============================================================================
#           04_inference_visual.ipynb: 纯视觉模型推理与提交
# =============================================================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from torchvision import transforms
from tqdm.notebook import tqdm

print("--- 步骤 1: 环境设置与模型定义 ---")

# --- 1. 定义纯视觉模型 VisualModel (ResNet18 版本) ---
# 必须与你训练时使用的结构完全一致
class VisualModel(nn.Module):
    def __init__(self, num_targets=5, pretrained=True):
        super(VisualModel, self).__init__()
        # 加载 ResNet18
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None)
        num_cnn_features = self.cnn.fc.in_features
        # 替换最后一层
        self.cnn.fc = nn.Sequential(
            nn.Linear(num_cnn_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_targets)
        )

    def forward(self, image):
        output = self.cnn(image)
        return output

# --- 2. 定义推理数据集类 ---
class InferenceDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 获取图片路径
        img_path_rel = self.df.iloc[idx]['image_path']
        img_path = os.path.join(self.image_dir, img_path_rel)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            # 容错处理
            image = Image.new('RGB', (224, 224), (0, 0, 0))
            
        if self.transform:
            image = self.transform(image)
            
        return image

# --- 3. 定义图像变换 (验证集标准) ---
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("环境设置完毕。\n")


# =============================================================================
# 步骤 2: 加载训练好的模型
# =============================================================================

print("--- 步骤 2: 加载模型 ---")

MODEL_PATH = 'best_visual_model.pth' 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 实例化模型
inference_model = VisualModel().to(device)

# 加载权重
if os.path.exists(MODEL_PATH):
    inference_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print(f"成功加载模型权重: {MODEL_PATH}")
else:
    raise FileNotFoundError(f"找不到模型文件: {MODEL_PATH}")

inference_model.eval()
print("模型已设置为评估模式。\n")


# =============================================================================
# 步骤 3: 准备测试数据
# =============================================================================

print("--- 步骤 3: 准备测试数据 ---")

test_df = pd.read_csv('csiro-biomass/test.csv')

# 去重：每张图片只预测一次
unique_images_df = test_df[['image_path']].drop_duplicates().reset_index(drop=True)

print(f"原始测试集行数: {len(test_df)}")
print(f"唯一图片数量: {len(unique_images_df)}")

# 创建数据集和加载器
IMAGE_DIR = './' 
test_dataset = InferenceDataset(unique_images_df, IMAGE_DIR, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("测试数据准备完毕。\n")


# =============================================================================
# 步骤 4: 执行预测
# =============================================================================

print("--- 步骤 4: 开始预测 ---")

all_predictions = []
target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']

with torch.no_grad():
    for images in tqdm(test_loader, desc="预测中"):
        images = images.to(device)
        
        # 预测 (Log 尺度)
        log_preds = inference_model(images)
        
        # 还原 (Exp 变换)
        preds = np.expm1(log_preds.cpu().numpy())
        
        all_predictions.append(preds)

# 合并结果
predictions_array = np.concatenate(all_predictions, axis=0)
pred_df_wide = pd.DataFrame(predictions_array, columns=target_cols)
result_df = pd.concat([unique_images_df, pred_df_wide], axis=1)

print("预测完成。\n")


# =============================================================================
# 步骤 5: 生成提交文件
# =============================================================================

print("--- 步骤 5: 生成提交文件 ---")

# 合并回原始 test.csv
final_df = pd.merge(test_df, result_df, on='image_path', how='left')

# 提取每一行对应的 target 值
def get_prediction(row):
    return row[row['target_name']] if row['target_name'] in row else 0.0

final_df['target'] = final_df.apply(get_prediction, axis=1)

# 保存
submission_df = final_df[['sample_id', 'target']]
submission_df.to_csv('submission.csv', index=False)

print("="*50)
print("提交文件 'submission.csv' 已成功生成！")
print(submission_df.head())
print("="*50)


--- 步骤 1: 环境设置与模型定义 ---
环境设置完毕。

--- 步骤 2: 加载模型 ---
成功加载模型权重: best_visual_model.pth
模型已设置为评估模式。

--- 步骤 3: 准备测试数据 ---
原始测试集行数: 5
唯一图片数量: 1
测试数据准备完毕。

--- 步骤 4: 开始预测 ---


/var/folders/mw/77ps8s1x14zg2_q9t7f74t4r0000gn/T/ipykernel_62095/3082563257.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inference_model.load_state_dict(torch.load(M

预测中:   0%|          | 0/1 [00:00<?, ?it/s]

预测完成。

--- 步骤 5: 生成提交文件 ---
提交文件 'submission.csv' 已成功生成！
                    sample_id    target
0  ID1001187975__Dry_Clover_g  1.249537
1    ID1001187975__Dry_Dead_g  2.135950
2   ID1001187975__Dry_Green_g  2.854519
3   ID1001187975__Dry_Total_g  6.423885
4         ID1001187975__GDM_g  4.913448
